In [83]:
import pandas as pd
compare_df = pd.read_csv('smilarity_result.csv')
compare_df[compare_df['Description choose columns'] > 145][['Question','SQL','Expect Columns','Description Min threshold','Description choose columns']]

,Question,SQL,Expect Columns,Description Min threshold,Description choose columns
33,What is number of times users open X store las...,SELECT COUNT(event_name) FROM team_tds.tds_int...,"['event_timestamp', 'event_name']",0.072696,146
34,How many users open X store last 7 days?,SELECT COUNT(DISTINCT user_pseudo_id) FROM tea...,"['user_pseudo_id', 'event_name', 'event_timest...",0.045698,163
36,How many users open X store last 30 days?,SELECT COUNT(DISTINCT user_pseudo_id) FROM tea...,"['user_pseudo_id', 'event_name', 'event_timest...",0.045825,162
37,How many users open X store more than once las...,SELECT COUNT(cnt) FROM\n (SELECT COUNT(DIST...,"['user_pseudo_id', 'event_name', 'event_timest...",0.041280,159
38,How many users open X store more than once las...,SELECT COUNT(cnt) FROM\n (SELECT COUNT(DIST...,"['user_pseudo_id', 'event_name', 'event_timest...",0.043333,156
40,How many users have opened X store more than 5...,SELECT COUNT(cnt) FROM\n (SELECT COUNT(DISTIN...,"['user_pseudo_id', 'event_name', 'event_timest...",0.067982,151
42,How many users have not used the app for more ...,SELECT COUNT(DISTINCT user_pseudo_id) FROM\n ...,"['user_pseudo_id', 'event_timestamp', 'event_n...",0.033596,147


In [8]:
from sentence_transformers import SentenceTransformer, models, InputExample, losses, util
from datasets import load_dataset
from torch.utils.data import DataLoader

model = SentenceTransformer('all-MiniLM-L6-v2')
# dataset_id = "embedding-data/QQP_triplets"
dataset_id = "embedding-data/sentence-compression"
dataset = load_dataset(dataset_id)

Found cached dataset json (/Users/thanawatthongpia/.cache/huggingface/datasets/embedding-data___json/embedding-data--sentence-compression-d643585deb6e0073/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

In [85]:
dataset['train']['set'][0]

["The USHL completed an expansion draft on Monday as 10 players who were on the rosters of USHL teams during the 2009-10 season were selected by the League's two newest entries, the Muskegon Lumberjacks and Dubuque Fighting Saints.",
 'USHL completes expansion draft']

In [86]:
train_examples = []
n_examples = dataset['train'].num_rows // 20
train_data = dataset['train']['set']

for i in range(n_examples):
  example = train_data[i]
  train_examples.append(InputExample(texts=[example[0], example[1]]))

In [87]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=64)
train_loss = losses.MultipleNegativesRankingLoss(model=model)

In [ ]:
num_epochs = 2
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [ ]:
len(train_dataloader)

In [ ]:
# model.fit(train_objectives=[(train_dataloader, train_loss)],
#           epochs=num_epochs,
#           warmup_steps=warmup_steps) 

In [1]:
from Description_base_linking import SchemaLinking

col = ['event_name', 'device_category', 'device_mobile_brand_name', 'device_mobile_model_name', 'device_mobile_marketing_name',
            'device_mobile_os_hardware_model', 'device_operating_system', 'device_operating_system_version', 'device_language', 'geo_continent',
            'geo_country', 'geo_region', 'geo_city', 'geo_sub_continent', 'geo_metro', 'app_info_version', 'app_info_install_source', 'platform',
            'firebase_previous_class', 'firebase_screen_class']
obj = SchemaLinking()
obj.add_cats_description(col)

In [2]:
print(obj.schema_description.loc[obj.schema_description['Column'] == 'event_name']['Description'].tolist())

['The name of the event.payatmcht_search, payatmcht_loction, payatmcht_category, virtualmall_detail_insufficient_point, user_engagement, pointx_more, pin_enter_landing, pointx_mypointx_bottom_bar, pointx_search, pointx_view_mypoint, pointx_home_bottom_bar, onboard_highlight_getstart, onboard_landing_scb_easy, session_start, settings_menu_landing, onboard_permission_notification_allow, onboard_lifestyle_landing, settings_delivery_addr_btn, settings_myacc, pointx_point_transfer_menu, virtualmall_detail_conditions_tab, pointx_dealoftheday_landing, myecoupon_valid_landing, mypointx_myecoupon, pointx_highlights_viewall, pointx_ecatalog, screen_view, pointx_payandmerchant_bottom_bar, app_remove, settings_security, delivery_addr_landing, scanToPay_listing_category, virtualmall_detail_soldout, payatmcht_detail, onboard_permission_notification_landing, mypointx_landing, payatmcht_confirm, pointx_highlight_landing, os_update, onboard_highlight_landing, payatmcht_landing, pointx_payatmerchant_men

In [4]:
# add event categories to event_name description
obj.ques_col_similarity("how many user remove app","event_name")

0.1140679121017456

In [5]:
# raw event_name description ("The name of the event")
obj = SchemaLinking()
obj.ques_col_similarity("how many user remove app","event_name")

-0.021025139838457108

In [1]:
from Description_base_linking import SchemaLinking
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from sentence_transformers import InputExample, losses
# import time

obj = SchemaLinking()
threshold = 0.2
compare_df = pd.read_csv('smilarity_result.csv')
questions = compare_df['Question'].to_list()
sql_queries = compare_df['SQL'].to_list()
train_ques_col = []

for i, q in enumerate(questions[:5]):
    print(q)
    expect_col = obj.columns_from_query(sql_queries[i])
    choose_col = obj.get_columns_threshold(q, threshold)

    # print("EXPECT COLUMNS:\t\t", expect_col)
    # print('SELECTED COLUMNS:\t', choose_col)

    expect_col_vec = obj.choose_col_onehot(expect_col)
    choose_col_vec = obj.choose_col_onehot(choose_col)
    accuracy = obj.accuracy_of_onehot(choose_col_vec, expect_col_vec)

    not_selected_col_idx =  (choose_col_vec == 0) & (expect_col_vec ==1 ).astype(int)
    not_selected_col = [ obj.schema_columns[idx] for idx in np.where(not_selected_col_idx == 1)[0]]
    if len(not_selected_col):
        print("COL NOT SELECTED:\t",not_selected_col)
        train_ques_col.append((q, not_selected_col))
    print(accuracy)
    print()

How many daily active users each day?
COL NOT SELECTED:	 ['event_date', 'ga_session_id']
0.5

How many monthly active users each month?
COL NOT SELECTED:	 ['user_pseudo_id', 'engagement_time_msec', 'ga_session_id']
0.25

What is the average number of daily active users last 7 days?
COL NOT SELECTED:	 ['event_date', 'user_pseudo_id', 'ga_session_id']
0.25

What is the mean number of daily active users from last 30 days?
COL NOT SELECTED:	 ['user_pseudo_id', 'ga_session_id']
0.5

When was the last time each user was active?
1.0



In [2]:
question = "How many daily active users each day?"

In [3]:
len(obj.get_columns_threshold(question))

25

In [4]:
obj.ques_col_similarity(question, 'event_date')

0.15984684228897095

In [8]:
obj.fine_tune_model(train_ques_col, 100)

Fine-tune model done!


In [9]:
len(obj.get_columns_threshold("How many daily active users each day?"))

26

In [10]:
obj.ques_col_similarity(question, 'event_date')

0.16712406277656555